In [163]:
import httpx
import pathlib
from decouple import config

In [164]:
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / "data"
GENERATED_DIR = DATA_DIR / "generated"
GENERATED_DIR.mkdir(exist_ok=True, parents=True)


In [165]:
prompt = input("What is your prompt?\n")
prompt_final = f"a photo of TOK adult man {prompt}"

In [166]:
prompt_final

'a photo of TOK adult man recording a video'

In [167]:
API_ACCESS_KEY = config('API_ACCESS_KEY')
BASE_URL = "http://127.0.0.1:8000"

headers = {
    "X-API-Key": API_ACCESS_KEY
}
endpoint = f"{BASE_URL}/generate"
res = httpx.post(endpoint, json={'prompt': prompt_final}, headers=headers)
res.json()

{'id': 'dx40raa141rma0cka4g9gqtsfr',
 'url': '/predictions/dx40raa141rma0cka4g9gqtsfr',
 'status': 'starting'}

In [168]:
import time 


pred_id = res.json().get('id')
pred_detail_endpoint = f"{BASE_URL}/predictions/{pred_id}"
res2 = httpx.get(pred_detail_endpoint, headers=headers)
pred_data = res2.json()
pred_status = pred_data.get('status')

max_attempts = 10
attempts = 0
ready_and_done = pred_status == 'succeeded'
while pred_status != 'succeeded' and max_attempts > attempts:
    _res = httpx.get(pred_detail_endpoint, headers=headers)
    pred_data = _res.json()
    pred_status = pred_data.get('status')
    print(pred_id, pred_status)
    if pred_status == "succeeded":
        ready_and_done = True
        break
    attempts += 1
    time.sleep(5)



dx40raa141rma0cka4g9gqtsfr processing
dx40raa141rma0cka4g9gqtsfr processing
dx40raa141rma0cka4g9gqtsfr processing
dx40raa141rma0cka4g9gqtsfr succeeded


In [169]:
def download_image(url, obj_id):
    fname = pathlib.Path(url).name
    outpath = GENERATED_DIR / obj_id / fname
    outpath.parent.mkdir(exist_ok=True, parents=True)
    if outpath.exists():
        return outpath
    res = httpx.get(url, headers=headers)
    res.raise_for_status()
    with open(outpath, 'wb') as f:
        f.write(res.content)
    return outpath

In [170]:
out_paths = []
if ready_and_done:
    files = pred_data.get('files')
    obj_id = pred_data.get('id')
    for file_path in files:
        endpoint = f"{BASE_URL}{file_path}"
        out_path = download_image(endpoint, obj_id)
        if out_path:
            out_paths.append(out_path)
out_paths

[PosixPath('/Users/cfe/Dev/superme-api/data/generated/dx40raa141rma0cka4g9gqtsfr/0.jpg'),
 PosixPath('/Users/cfe/Dev/superme-api/data/generated/dx40raa141rma0cka4g9gqtsfr/1.jpg')]

In [171]:
from PIL import Image

for path in out_paths:
    img = Image.open(path)
    img.show()